## Getting Started

Attempted walkthrough of the [getting started](https://docs.aws.amazon.com/kinesisanalytics/latest/java/get-started-exercise.html) tutorial for KDA-J

In [ ]:
import boto3

kda_client = boto3.client('kinesisanalyticsv2')
kinesis = boto3.client('kinesis')
iam = boto3.client('iam')
kda2 = boto3.client('kinesisanalyticsv2')

### Input and Output Streams

In [ ]:
cis = kinesis.create_stream(
    StreamName='ExampleInputStream',
    ShardCount=1
)

print(cis)

In [ ]:
cos = kinesis.create_stream(
    StreamName='ExampleOutputStream',
    ShardCount=1
)

print(cos)

### Input Generator

In [ ]:
import datetime
import json
import random
import boto3

def get_data():
    return {
        'EVENT_TIME': datetime.datetime.now().isoformat(),
        'TICKER': random.choice(['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']),
        'PRICE': round(random.random() * 100, 2)}


def generate(stream_name, kinesis_client):
    while True:
        data = get_data()
        print(data)
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey="partitionkey")

In [ ]:
# Run it
generate("ExampleInputStream", kinesis)

### Application Code File

Compile the getting started code, create a bucket, and upload it to the bucket.

e.g.

```
mvn package
aws s3api create-bucket --bucket dskdaj-getting-started
aws s3 cp target/aws-kinesis-analytics-java-apps-1.0.jar s3://dskdaj-getting-started/getting-started.jar
```

In [ ]:
code_bucket='dskdaj-getting-started'
jar_key='getting-started.jar'

### IAM Role and Policy

Here's the generated policy as the starting point for an app named scratch.

The kinesis-analytics-scratch-us-east-1 role `via aws iam get-role --role-name kinesis-analytics-scratch-us-east-1`

```
{
    "Role": {
        "Path": "/service-role/",
        "RoleName": "kinesis-analytics-scratch-us-east-1",
        "RoleId": "xxx",
        "Arn": "arn:aws:iam::xxx:role/service-role/kinesis-analytics-scratch-us-east-1",
        "CreateDate": "2021-07-03T16:11:50+00:00",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "kinesisanalytics.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        },
        "MaxSessionDuration": 3600,
        "RoleLastUsed": {}
    }
}
```
Policies:

```
aws iam list-attached-role-policies  --role-name kinesis-analytics-scratch-us-east-1
{
    "AttachedPolicies": [
        {
            "PolicyName": "kinesis-analytics-service-scratch-us-east-1",
            "PolicyArn": "arn:aws:iam::xxx:policy/service-role/kinesis-analytics-service-scratch-us-east-1"
        }
    ]
}
```

```
aws iam get-policy --policy-arn arn:aws:iam::xxx:policy/service-role/kinesis-analytics-service-scratch-us-east-1
{
    "Policy": {
        "PolicyName": "kinesis-analytics-service-scratch-us-east-1",
        "PolicyId": "xxx",
        "Arn": "arn:aws:iam::xxx:policy/service-role/kinesis-analytics-service-scratch-us-east-1",
        "Path": "/service-role/",
        "DefaultVersionId": "v1",
        "AttachmentCount": 1,
        "PermissionsBoundaryUsageCount": 0,
        "IsAttachable": true,
        "CreateDate": "2021-07-03T16:11:50+00:00",
        "UpdateDate": "2021-07-03T16:11:50+00:00",
        "Tags": []
    }
}
```

aws iam get-policy-version --policy-arn arn:aws:iam::xxx:policy/service-role/kinesis-analytics-service-sample-us-east-1 --version-id v1

```
{
    "PolicyVersion": {
        "Document": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "ReadCode",
                    "Effect": "Allow",
                    "Action": [
                        "s3:GetObject",
                        "s3:GetObjectVersion"
                    ],
                    "Resource": [
                        "arn:aws:s3:::kinesis-analytics-placeholder-s3-bucket/kinesis-analytics-placeholder-s3-object"
                    ]
                },
                {
                    "Sid": "ListCloudwatchLogGroups",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogGroups"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:xxx:log-group:*"
                    ]
                },
                {
                    "Sid": "ListCloudwatchLogStreams",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogStreams"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:xxx:log-group:/aws/kinesis-analytics/sample:log-stream:*"
                    ]
                },
                {
                    "Sid": "PutCloudwatchLogs",
                    "Effect": "Allow",
                    "Action": [
                        "logs:PutLogEvents"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:xxx:log-group:/aws/kinesis-analytics/sample:log-stream:kinesis-analytics-log-stream"
                    ]
                }
            ]
        },
        "VersionId": "v1",
        "IsDefaultVersion": true,
        "CreateDate": "2021-07-04T15:21:25+00:00"
    }
}
```


In [ ]:
assume_role_policy_document="""{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "kinesisanalytics.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
"""

In [ ]:
cr = iam.create_role(
    RoleName='kda-sample-role',
    Path='/service-role/',
    AssumeRolePolicyDocument=assume_role_policy_document
)

print(cr)

In [ ]:
import os
account_no = os.environ['PRODUCER_ACCOUNT_NO']

In [ ]:
policy_document="""{
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "ReadCode",
                    "Effect": "Allow",
                    "Action": [
                        "s3:GetObject",
                        "s3:GetObjectVersion"
                    ],
                    "Resource": [
                        "arn:aws:s3:::dskdaj-getting-started/*"
                    ]
                },
                {
                    "Sid": "ListCloudwatchLogGroups",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogGroups"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:""" + account_no + """:log-group:*"
                    ]
                },
                {
                    "Sid": "ListCloudwatchLogStreams",
                    "Effect": "Allow",
                    "Action": [
                        "logs:DescribeLogStreams"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:""" + account_no + """:log-group:/aws/kinesis-analytics/sample:log-stream:*"
                    ]
                },
                {
                    "Sid": "PutCloudwatchLogs",
                    "Effect": "Allow",
                    "Action": [
                        "logs:PutLogEvents"
                    ],
                    "Resource": [
                        "arn:aws:logs:us-east-1:""" + account_no + """:log-group:/aws/kinesis-analytics/sample:log-stream:kinesis-analytics-log-stream"
                    ]
                },
                {
                
                    "Sid": "ReadInputStream",
                    "Effect": "Allow",
                    "Action": [
                        "kinesis:*"
                    ],
                    "Resource": [
                        "arn:aws:kinesis:us-east-1:""" + account_no + """:stream/ExampleInputStream"
                    ]
                },
                {
                    "Sid": "WriteOutputStream",
                    "Effect": "Allow",
                    "Action": "kinesis:*",
                    "Resource": "arn:aws:kinesis:us-east-1:""" + account_no + """:stream/ExampleOutputStream"
                }
            ]
}"""

In [ ]:
print(policy_document)

In [ ]:
cp = iam.create_policy(
    PolicyName='kda-sample-app',
    Path='/service-role/',
    PolicyDocument=policy_document
)

print(cp)

In [ ]:
ap = iam.attach_role_policy(
    RoleName='kda-sample-role',
    PolicyArn="arn:aws:iam::{}:policy/service-role/kda-sample-app".format(account_no)
)

### Application Definition

Details from creating the application in the console then dumping it via describe_application

```
{
   "ApplicationDetail":{
      "ApplicationARN":"arn:aws:kinesisanalytics:us-east-1:xxx:application/sample",
      "ApplicationName":"sample",
      "RuntimeEnvironment":"FLINK-1_11",
      "ServiceExecutionRole":"arn:aws:iam::xxx:role/service-role/kda-sample-role",
      "ApplicationStatus":"RUNNING",
      "ApplicationVersionId":2,
      "CreateTimestamp":{
         "$date":1625415087000
      },
      "LastUpdateTimestamp":{
         "$date":1625415425000
      },
      "ApplicationConfigurationDescription":{
         "ApplicationCodeConfigurationDescription":{
            "CodeContentType":"ZIPFILE",
            "CodeContentDescription":{
               "CodeMD5":"f17ccaa0d429ecd9856959fd305a3fb2",
               "CodeSize":44129741,
               "S3ApplicationCodeLocationDescription":{
                  "BucketARN":"arn:aws:s3:::dskdaj-getting-started",
                  "FileKey":"getting-started.jar"
               }
            }
         },
         "RunConfigurationDescription":{
            "ApplicationRestoreConfigurationDescription":{
               "ApplicationRestoreType":"SKIP_RESTORE_FROM_SNAPSHOT"
            },
            "FlinkRunConfigurationDescription":{
               "AllowNonRestoredState":false
            }
         },
         "FlinkApplicationConfigurationDescription":{
            "CheckpointConfigurationDescription":{
               "ConfigurationType":"DEFAULT",
               "CheckpointingEnabled":true,
               "CheckpointInterval":60000,
               "MinPauseBetweenCheckpoints":5000
            },
            "MonitoringConfigurationDescription":{
               "ConfigurationType":"CUSTOM",
               "MetricsLevel":"APPLICATION",
               "LogLevel":"INFO"
            },
            "ParallelismConfigurationDescription":{
               "ConfigurationType":"CUSTOM",
               "Parallelism":1,
               "ParallelismPerKPU":1,
               "CurrentParallelism":1,
               "AutoScalingEnabled":true
            }
         },
         "EnvironmentPropertyDescriptions":{
            "PropertyGroupDescriptions":[
               {
                  "PropertyGroupId":"ProducerConfigProperties",
                  "PropertyMap":{
                     "AggregationEnabled":"false",
                     "aws.region":"us-east-1",
                     "flink.inputstream.initpos":"LATEST"
                  }
               }
            ]
         },
         "ApplicationSnapshotConfigurationDescription":{
            "SnapshotsEnabled":false
         }
      },
      "CloudWatchLoggingOptionDescriptions":[
         {
            "CloudWatchLoggingOptionId":"1.1",
            "LogStreamARN":"arn:aws:logs:us-east-1:xxx:log-group:/aws/kinesis-analytics/sample:log-stream:kinesis-analytics-log-stream"
         }
      ]
   },
   "ResponseMetadata":{
      "RequestId":"7348d1f9-09da-44cd-b99a-bea89708de73",
      "HTTPStatusCode":200,
      "HTTPHeaders":{
         "x-amzn-requestid":"7348d1f9-09da-44cd-b99a-bea89708de73",
         "content-type":"application/x-amz-json-1.1",
         "content-length":"2153",
         "date":"Sun, 04 Jul 2021 16:40:54 GMT"
      },
      "RetryAttempts":0
   }
}

```

In [ ]:
kca = kda2.create_application(
    ApplicationName='sample',
    ApplicationDescription='Sample getting started application',
    RuntimeEnvironment='FLINK-1_11',
    ServiceExecutionRole='arn:aws:iam::{}:role/service-role/kda-sample-role'.format(account_no),
    ApplicationConfiguration={
        'ApplicationCodeConfiguration': {
            'CodeContent': {
                'S3ContentLocation': {
                    'BucketARN': 'arn:aws:s3:::' + code_bucket,
                    'FileKey': jar_key,
                }
            },
            'CodeContentType': 'ZIPFILE'
        },
        'FlinkApplicationConfiguration': {
            'CheckpointConfiguration': {
                'ConfigurationType': 'DEFAULT'
            },
            'MonitoringConfiguration': {
                'ConfigurationType': 'CUSTOM',
                'MetricsLevel': 'APPLICATION',
                'LogLevel': 'DEBUG'
            },
            'ParallelismConfiguration': {
                'ConfigurationType': 'CUSTOM',
                'Parallelism': 1,
                'ParallelismPerKPU': 1,
                'AutoScalingEnabled': True
            }
        },
        'EnvironmentProperties': {
            'PropertyGroups': [
                {
                    "PropertyGroupId":"ProducerConfigProperties",
                      "PropertyMap":{
                         "AggregationEnabled":"false",
                         "aws.region":"us-east-1",
                         "flink.inputstream.initpos":"LATEST"
                      }
                }
            ]
        },
        'ApplicationSnapshotConfiguration': {
            'SnapshotsEnabled': False
        }
        
    },
    CloudWatchLoggingOptions=[
        {
            "LogStreamARN":"arn:aws:logs:us-east-1:{}:log-group:/aws/kinesis-analytics/sample:log-stream:kinesis-analytics-log-stream".format(account_no)
        }
    ]
    
)
print(kca)

In [ ]:
sao = kda2.start_application(
    ApplicationName='sample',
    RunConfiguration={
        'FlinkRunConfiguration': {
            'AllowNonRestoredState': False
        },
        'ApplicationRestoreConfiguration': {
            'ApplicationRestoreType': 'SKIP_RESTORE_FROM_SNAPSHOT'
        }
    } 
)
print(sao)

In [ ]:
# List applications and see their states...
kda2.list_applications()

### Read From Output

In [ ]:
shards = kinesis.list_shards(
    StreamName='ExampleOutputStream'
)
print(shards)

In [ ]:
itor = kinesis.get_shard_iterator(
    StreamName='ExampleOutputStream',
    ShardIteratorType='TRIM_HORIZON',
    ShardId=shards['Shards'][0]['ShardId']
)
print(itor)

In [ ]:
kinesis.get_records(
    ShardIterator=itor['ShardIterator']
)

## Clean Up

#### Application

In [ ]:
kda2.stop_application(
    ApplicationName='sample',
    Force=True
)

In [ ]:
da = kda2.describe_application(
    ApplicationName='sample'
)
create_timestamp = da['ApplicationDetail']['CreateTimestamp']

In [ ]:
from bson import json_util
import json

json.dumps(da, default=json_util.default)

In [ ]:
summaries = kda2.list_applications()['ApplicationSummaries']
print(summaries)

In [ ]:
kda2.delete_application(
    ApplicationName='sample',
    CreateTimestamp=create_timestamp
)

#### IAM

In [ ]:
policy_arn="arn:aws:iam::{}:policy/service-role/kda-sample-app".format(account_no)
print(policy_arn)

In [ ]:
iam.detach_role_policy(
    RoleName='kda-sample-role',
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_policy(
    PolicyArn=policy_arn
)

In [ ]:
iam.delete_role(
    RoleName='kda-sample-role'
)

#### Streams

In [ ]:
kinesis.delete_stream(
    StreamName='ExampleInputStream'
)

In [ ]:
kinesis.delete_stream(
    StreamName='ExampleOutputStream'
)

In [ ]:
kinesis.list_streams()